Nome Rm Email

In [2]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install galois


[notice] A new release of pip is available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Algoritmo de Simon
O algoritmo de Simon é um problema e um algoritmo quântico que visa encontrar uma string secreta oculta em uma função booleana. Abaixo está uma explicação detalhada das partes do código relacionado à implementação do algoritmo de Simon em computação quântica.

## Função simon_function(s)
Esta função cria um circuito quântico que implementa uma porta de consulta (query gate) para o problema de Simon. A string s passada como argumento é a string secreta que o algoritmo tentará encontrar.

O circuito quântico tem um total de 2n qubits, onde n é o comprimento da string s. Uma permutação aleatória pi é gerada para esconder a string s. Em seguida, é criada uma matriz query_gate que representa a função f(x) como definida no problema de Simon. Essa função é composta por g(x) = min{x, x ^ s}, onde ^ denota a operação XOR. O circuito quântico consiste apenas na aplicação da matriz query_gate aos qubits.

In [3]:
# import random #é usado para gerar uma permutação aleatória.
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit
import numpy as np


def simon_function(s: str):
    """
    Cria um QuantumCircuit que implementa uma porta de consulta para o problema de Simon, obedecendo à promessa da string oculta `s`
    """
    # Nosso circuito quântico tem 2n qubits, onde n = len(s)
    n = len(s)
    qc = QuantumCircuit(2 * n)

    # Define uma permutação aleatória de todas as strings de n bits. Essa permutação efetivamente oculta a string s.
    pi = np.random.permutation(2**n)

    # Agora vamos definir uma porta de consulta explicitamente. A ideia é primeiro definir uma função g(x) = min{x, x ^ s}, que
    # é uma função simples que atende à promessa, e então tomamos f como a composição de g e a permutação aleatória pi. Isso nos dá uma função aleatória que atende à promessa para s.

    query_gate = np.zeros((4**n, 4**n))
    for x in range(2**n):
        for y in range(2**n):
            z = y ^ pi[min(x, x ^ int(s, 2))]
            query_gate[x + 2**n * z, x + 2**n * y] = 1

    # Nosso circuito consiste apenas nessa única porta de consulta
    qc.unitary(query_gate, range(2 * n))
    return qc

## Função simon_measurements(problem, k)
Esta função realiza a parte quântica do algoritmo de Simon. Ela recebe como entrada um circuito quântico problem e um número inteiro k. A função cria um novo circuito quântico que realiza as etapas do algoritmo, incluindo a aplicação de portas Hadamard, a composição do circuito problem, a aplicação de mais portas Hadamard e, finalmente, a medição dos qubits. A função utiliza o simulador Aer do Qiskit para executar o circuito qc k vezes e retornar os resultados das medições em formato de memória.

In [4]:
from qiskit_aer import AerSimulator
from qiskit import ClassicalRegister


def simon_measurements(problem: QuantumCircuit, k: int):
    """
    Parte quântica do algoritmo de Simon. Dado um `QuantumCircuit` que
    implementa f, obtenha `k` medições para serem processadas posteriormente.
    """
    n = problem.num_qubits // 2

    # Cria um circuito quântico com 2n qubits e n qubits de medição.
    qc = QuantumCircuit(2 * n, n)
    
    # Aplica portas Hadamard aos primeiros n qubits.
    qc.h(range(n))
    
    # Componha o circuito com o problema especificado.
    qc.compose(problem, inplace=True)
    
    # Aplica portas Hadamard novamente aos primeiros n qubits.
    qc.h(range(n))
    
    # Realiza medições nos primeiros n qubits e associa-os aos n qubits de medição.
    qc.measure(range(n), range(n))

    # Executa o circuito no simulador Aer do Qiskit com k disparos (shots) e mantém a memória ativada.
    result = AerSimulator().run(qc, shots=k, memory=True).result()
    
    # Retorna os resultados da memória (resultados individuais das medições).
    return result.get_memory()

In [5]:
simon_measurements(
    simon_function("11011"),
    k=12
)

['10001',
 '11011',
 '00100',
 '11000',
 '11000',
 '10001',
 '10010',
 '00000',
 '00000',
 '01110',
 '11100',
 '11100']

## Função simon_algorithm(problem)
Esta função é responsável por executar o algoritmo de Simon completo. Recebe como entrada o circuito problem, que deve ser uma porta de consulta criada pela função simon_function. Primeiro, a função chama simon_measurements para obter as medições do circuito quântico, e os resultados são armazenados na variável measurements. Em seguida, faz o seguinte processamento clássico dos resultados:

Converte as medições de strings de bits para uma matriz 2D de inteiros.
Usa operações de álgebra linear em GF(2) (corpo finito de ordem 2) para encontrar o espaço nulo (null space) da matriz.
Converte a solução do espaço nulo de volta para uma string binária, que representa a string secreta s.
Finalmente, a função retorna a string binária s.

In [6]:
import numpy as np
import galois


def simon_algorithm(problem: QuantumCircuit):
    """
    Dado um `QuantumCircuit` que implementa uma porta de consulta para o problema de Simon, retorna a string oculta `s`.
    """

    # Parte quântica: execute o circuito definido anteriormente k vezes e colete os resultados das medições.
    # Substitua +10 por +r para qualquer número inteiro não negativo r, dependendo da confiança desejada.

    measurements = simon_measurements(problem, k=problem.num_qubits // 2 + 10)
    print("Resultados das medições:")
    display(measurements)

    # Pós-processamento clássico:

    # 1. Converta as medições no formato '11101' para uma matriz 2D de inteiros
    matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)

    # 2. Interprete a matriz usando aritmética módulo 2 e encontre o espaço nulo
    null_space = galois.GF(2)(matrix).null_space()
    print("Espaço nulo:")
    display(null_space)

    # 3. Converta de volta para uma string
    print("Palpite para a string oculta s:")
    if len(null_space) == 0:
        # Sem solução não trivial; `s` é composta apenas por zeros
        return "0" * len(measurements[0])
    return "".join(np.array(null_space[0]).astype(str))

In [7]:
simon_algorithm(
    simon_function("10011")
)

Measurement results:


['01100',
 '00100',
 '01000',
 '10010',
 '00000',
 '00111',
 '10001',
 '01111',
 '10001',
 '10001',
 '10010',
 '00111',
 '01000',
 '01111',
 '11010']

Null space:


GF([[1, 0, 0, 1, 1]], order=2)

Guess for hidden string s:


'10011'

# Algoritmo de Simon em Computação Quântica

O algoritmo de Simon é um problema e um algoritmo quântico que tem como objetivo encontrar uma string secreta oculta em uma função booleana. Vamos explicar o funcionamento deste algoritmo, incluindo a parte matemática.

## Pré-requisitos

- Compreensão básica de computação quântica.
- Familiaridade com álgebra linear e operações em corpos finitos.

## Etapas do Algoritmo

O algoritmo de Simon pode ser dividido em três etapas principais:

### 1. Parte Quântica

A parte quântica do algoritmo envolve a criação de um circuito quântico que implementa uma porta de consulta (query gate) para o problema de Simon. A ideia é definir uma função `f(x)` que satisfaz a promessa do problema, onde `x` é uma entrada de n bits. Isso é feito da seguinte forma:

- Um circuito quântico é criado com 2n qubits, onde `n` é o comprimento da string secreta que estamos tentando encontrar.
- Uma permutação aleatória de todas as strings de `n` bits é gerada para efetivamente esconder a string secreta.
- A função `f(x)` é definida como a composição de duas funções: `g(x)` e a permutação aleatória.
- A função `g(x)` é definida como `min{x, x ^ s}`, onde `^` denota a operação XOR.

### 2. Medição Quântica

O circuito quântico é então executado várias vezes (geralmente muitas vezes mais do que o comprimento da string secreta), e as medições são coletadas. O número de execuções do circuito depende da confiança desejada na determinação da string secreta.

### 3. Pós-processamento Clássico

Depois de coletar as medições, a parte clássica do algoritmo envolve o seguinte processamento:

- As medições são convertidas em uma matriz 2D de inteiros.
- A matriz é interpretada como uma operação de aritmética modular 2, e o espaço nulo (null space) é calculado.
- A solução do espaço nulo é convertida de volta em uma string binária, que representa a string secreta `s`.

## Conclusão

O algoritmo de Simon é um exemplo de como a computação quântica pode ser usada para resolver problemas de forma mais eficiente do que abordagens clássicas. Ele aproveita as propriedades quânticas, como a sobreposição e a interferência, para encontrar a string secreta em uma função booleana em um número significativamente menor de iterações do que seria necessário em computação clássica.

Espero que esta explicação tenha esclarecido o funcionamento do algoritmo de Simon, incluindo a parte matemática envolvida.
